In [ ]:
from pyquery import PyQuery as pq
def getDBLP(URL):
    doc=pq(url=URL)
    det= doc(".visit.drop-down")
    data={'site':det('.head a').attr('href')}
    
    links=det('.body a')
    for l in links:
        text=pq(l).text()
        if(text):
            data[text]=l.attrib['href']
    if len(links)<3:
        all=[doc('#index-side-panel').next().find('a'),
        doc('#info-section').next().find('a')]
        
        for item in all:
            if(item):
                data[item.text()]=item.attr('href')
        
        
    if not(data['site']): 
        for p in data:
            if(data[p]):
                data['site']=data[p]
                break
    return data
import json
all=json.load(open('core-jnls.json'))

# getDBLP('https://dblp.uni-trier.de/db/conf/ant/index.html')

In [ ]:
i=0
import time
for item in all:
    i+=1
    if(item['DBLP Source'] and not('Links' in item)):
#         print(item['Title'])
        try:
            item['Links']=getDBLP(item['DBLP Source'])
            print(f"{i}\t{item['Title']}==== {item['Links']['site']}")
        except:
            print(f'error {item["Title"]}')
        time.sleep(1)
#         break


In [ ]:
with open('core-jnls.json', 'w') as fp:
    json.dump(all, fp)

In [ ]:
#!pip install pyquery
from pyquery import PyQuery as pq
import json

def getCoreData(conf, id):
    if(conf):
        TYPE = 'Conference'
        URL = f'http://portal.core.edu.au/conf-ranks/{id}'
    else:
        TYPE = 'Journal'
        URL = f'http://portal.core.edu.au/jnl-ranks/{id}'
    doc=pq(url=URL)
    det= doc("#detail")
    
    
    data={
    'CoreId':id,
    'Type':TYPE,
    'Title':det('h2:first').text(),
    'Acronym':'',
    'Acronym2':'',
    'DBLP Source':'',
    'Rank':'',
    'Rank Source':'',
    'Field':[],
    'Info':[]    
    }

    details=det('.detail')
    for i in range(len(details)):
        rows=details.eq(i)('.row')
        row=rows.eq(0)
        splt=row.text().split(':')
        titr=splt[0].strip()
        val=row.text()[len(splt[0])+1:].strip()

        if titr=='Acronym':
            data[titr]=val
        elif titr=='DBLP Source':
            if(val!="N/A"):
                data[titr]=val
                data['Acronym2']=val.split('/')[-1]
        elif titr=='Source':
            info={'Source':'','Rank':'','Field Of Research':[]}
            for j in range(len(rows)):
                row=rows.eq(j)
                splt=row.text().split(':')
                titr=splt[0].strip()
                val=row.text()[len(splt[0])+1:].strip()

                if titr=='Field Of Research':
                    val=val.replace('†','')
                    info['Field Of Research'].append(val)                
                    if not (val in data['Field']): data['Field'].append(val)
                elif val=='':
                    info['other']=row.html().strip().replace('\n','').replace('  ','').replace('a href="/core','a href="http://portal.core.edu.au/core')
                else:
                    if(titr=='Rank' and data['Rank']==''): 
                        data['Rank']=val
                        data['Rank Source']=info['Source']
                    info[titr]=val
            data['Info'].append(info)

        
        if not(conf):
            doc=pq(url=f'http://portal.core.edu.au/jnl-ranks/?search={data["Title"]}&by=all&source=all&sort=atitle&page=1')
            det= doc("table tr td a").attr('href')
            if not (det == None):
                data['DBLP Source']=det
                data['Acronym2']=det.split('/')[-1]
            
           
    return data

confs=[]
for i in range(2302):
    print(i,end=' ')
    try:
        confs.append(getCoreData(conf=True,id=i))
    except Exception as e:
        print(e)

with open('core-conf2021.json', 'w') as fp:
    json.dump(confs, fp)

# jnls=[]
# for i in range(937):
#     print(i,end=' ')
#     try:
#         jnls.append(getCoreData(conf=False,id=i))
#     except Exception as e:
#         print(e)
    

# with open('core-jnls2021.json', 'w') as fp:
#     json.dump(jnls, fp)    
    
# import csv
# with open('core2020.tsv', 'wt') as out_file:
#     tsv_writer = csv.writer(out_file, delimiter='\t')
#     tsv_writer.writerow(['name', 'field'])
    
    


In [12]:
import pandas as pd
def get_scimagojr(year=2019):
    info=json.load(open('scimagojr_info.json'))
    print(info['area']['1000'])
    url=f'https://www.scimagojr.com/shapeofscience/ajaxmap.php?authorcountry=0&year={year}'
    df=pd.read_csv(url,delimiter=';')
    data={}
    for k,row in df.iterrows():
#         cats=row['CATEGORIAS'].split(',')
#         areas=row['AREAS'].split(',')
        
#         obj={
#             'Id':row['SOURCEID'],
#             'Title':row['SOURCETITLE'],
            
#             'SJR':row['SJR'],
            
#         }
        forbiden=['posX','posY','Cluster','wDegree','CATEGORIAS','AREAS','SCIELO']
        obj={k:row[k] for k in row.keys() if not(k in forbiden)}
        obj['Topic']={y:[x for x in row['CATEGORIAS'].split(',') if x[0:2]==y[0:2]] for y in row['AREAS'].split(',')}
        
        data[row['SOURCEID']]=obj
        
    with open(f'scimagojr_{year}.json', 'w') as fp:
        json.dump(data, fp)
    return pd.DataFrame.from_dict(data, orient='index')
a=get_scimagojr(2022)
a

Multidisciplinary


,SOURCEID,SOURCETITLE,CATEGORIA,AREA,COUNTRY,SJR,CUARTIL,ARTICULOS_YEAR,REFERENCIAS_YEAR,CITAS_TOTALES,...,ITEMS_MULTIPAIS,AUTOCITAS_VENTANA,ITEMS_CONCITA_VENTANA,ITEMS_SINCITA_VENTANA,ITEMS_TOTALES_VENTANA,ITEMS_RATIOCONCITA_VENTANA,ITEMS_RATIOSINCITA_VENTANA,H,OPENACCESS,Topic
21206,21206,Nature,1000,1000,GBR,20.957,1,3132,67627,168543,...,827,16374,4900,2567,7467,0.656,0.344,1331,0,{'1000': ['1000']}
23571,23571,Science,1207,1200,USA,13.328,1,2066,40993,130524,...,619,10654,4309,4964,9273,0.465,0.535,1283,0,"{'1000': ['1000'], '1200': ['1207']}"
15847,15847,New England Journal of Medicine,2701,2700,USA,26.015,1,1410,14396,133956,...,329,4029,2810,1751,4561,0.616,0.384,1130,0,{'2700': ['2701']}
18434,18434,Cell,1301,1300,USA,26.494,1,420,27609,67791,...,157,7437,1466,171,1637,0.896,0.104,856,0,{'1300': ['1301']}
16590,16590,"Lancet, The",2701,2700,GBR,14.607,1,1198,20483,80420,...,383,5827,2526,1875,4401,0.574,0.426,855,0,{'2700': ['2701']}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21101089624,21101089624,Quaderni del '900,1208,1200,ITA,0.102,3,10,384,0,...,1,1,0,20,20,0.000,1.000,0,0,"{'1200': ['1208'], '3300': ['3310']}"
21101091058,21101091058,La Palabra,1208,1200,COL,0.101,3,29,834,0,...,1,10,0,20,20,0.000,1.000,0,0,{'1200': ['1208']}
21101093542,21101093542,Estudios de Historia de Espana,1202,1200,ARG,0.103,4,10,414,0,...,1,0,0,11,11,0.000,1.000,0,0,"{'1200': ['1201', '1202', '1212']}"
21101096409,21101096409,Boletin del Museo del Prado,1213,1200,ESP,0.102,4,11,506,0,...,0,4,0,18,18,0.000,1.000,0,0,"{'1200': ['1209', '1213']}"


In [13]:
import pandas as pd
import numpy as np
def get_scimagojr_journal(year=2020):
    info=json.load(open('scimagojr_info.json'))
    
    url=f'https://www.scimagojr.com/journalrank.php?year={year}&type=j&area=1700&out=xls'
    df=pd.read_csv(url,delimiter=';')
    data=[]
    for k,row in df.iterrows():
#         cats=row['CATEGORIAS'].split(',')
#         areas=row['AREAS'].split(',')
        
#         obj={
#             'Id':row['SOURCEID'],
#             'Title':row['SOURCETITLE'],
            
#             'SJR':row['SJR'],
            
#         }
        forbiden=['posX','posY','Cluster','wDegree','CATEGORIAS','AREAS','SCIELO']
        obj={k:row[k] if row[k]==row[k] else '' for k in row.keys() if not(k in forbiden)}
#         obj['Topic']={y:[x for x in row['CATEGORIAS'].split(',') if x[0:2]==y[0:2]] for y in row['AREAS'].split(',')}
#         print(obj)
        data.append(obj)
        
    with open(f'scimagojr_journal_{year}.json', 'w') as fp:
        json.dump(data, fp)
    return data;
a=get_scimagojr_journal(2022)
a


[{'Rank': 1,
  'Sourceid': 19300156903,
  'Title': 'Foundations and Trends in Machine Learning',
  'Type': 'journal',
  'Issn': '19358245, 19358237',
  'SJR': '13,775',
  'SJR Best Quartile': 'Q1',
  'H index': 36,
  'Total Docs. (2022)': 2,
  'Total Docs. (3years)': 15,
  'Total Refs.': 173,
  'Total Cites (3years)': 851,
  'Citable Docs. (3years)': 15,
  'Cites / Doc. (2years)': '39,10',
  'Ref. / Doc.': '86,50',
  'Country': 'United States',
  'Region': 'Northern America',
  'Publisher': 'Now Publishers Inc',
  'Coverage': '2008-2022',
  'Categories': 'Artificial Intelligence (Q1); Human-Computer Interaction (Q1); Software (Q1)',
  'Areas': 'Computer Science'},
 {'Rank': 2,
  'Sourceid': 18902,
  'Title': 'IEEE Journal on Selected Areas in Communications',
  'Type': 'journal',
  'Issn': '07338716',
  'SJR': '7,656',
  'SJR Best Quartile': 'Q1',
  'H index': 251,
  'Total Docs. (2022)': 242,
  'Total Docs. (3years)': 685,
  'Total Refs.': 11676,
  'Total Cites (3years)': 11104,
  'Ci

In [14]:
import numpy as np
np.nan=='a'

False

In [15]:
import pandas as pd
import numpy as np
def update_citescore_journal():
    info=json.load(open('scimagojr_journal_2022.json'))
    for d in info:
        d['CiteScore']=0
        d['SNIF']=0
#         d['Percentile']=0
    idm={d['Sourceid']:d for d in info}
    
    url=f'citescore.csv'
    df=pd.read_csv(url)
    data=[]
    for k,row in df.iterrows():
#         cats=row['CATEGORIAS'].split(',')
#         areas=row['AREAS'].split(',')
        id=row['Scopus Source ID']
        
        if(id in idm):
#             print(row)
            d=idm[id]
            d['CiteScore']=row['CiteScore 2020']
            d['SNIP']=row['SNIP']
#             d['Percentile']=row['Percentile']
        else: 
#             print(row)
            pass
            
#         obj={
#             'Id':row['SOURCEID'],
#             'Title':row['SOURCETITLE'],
            
#             'SJR':row['SJR'],
            
#         }
        
    with open(f'scimagojr_journal_2022_cs.json', 'w') as fp:
        json.dump(info, fp)
    return info;
a=update_citescore_journal()
a


[{'Rank': 1,
  'Sourceid': 19300156903,
  'Title': 'Foundations and Trends in Machine Learning',
  'Type': 'journal',
  'Issn': '19358245, 19358237',
  'SJR': '13,775',
  'SJR Best Quartile': 'Q1',
  'H index': 36,
  'Total Docs. (2022)': 2,
  'Total Docs. (3years)': 15,
  'Total Refs.': 173,
  'Total Cites (3years)': 851,
  'Citable Docs. (3years)': 15,
  'Cites / Doc. (2years)': '39,10',
  'Ref. / Doc.': '86,50',
  'Country': 'United States',
  'Region': 'Northern America',
  'Publisher': 'Now Publishers Inc',
  'Coverage': '2008-2022',
  'Categories': 'Artificial Intelligence (Q1); Human-Computer Interaction (Q1); Software (Q1)',
  'Areas': 'Computer Science',
  'CiteScore': 37.8,
  'SNIF': 0,
  'SNIP': 17.355},
 {'Rank': 2,
  'Sourceid': 18902,
  'Title': 'IEEE Journal on Selected Areas in Communications',
  'Type': 'journal',
  'Issn': '07338716',
  'SJR': '7,656',
  'SJR Best Quartile': 'Q1',
  'H index': 251,
  'Total Docs. (2022)': 242,
  'Total Docs. (3years)': 685,
  'Total R

In [20]:
import pandas as pd
import numpy as np
from pyquery import PyQuery as pq

def update_link_journal():
    info=json.load(open('scimagojr_journal_2022_cs.json'))
    try:
#         for d in info:
#             d['Scope']=''
        for d in info:
            if(d.get('Scope','')!=''):continue
            print(f"{d['Sourceid']}:{d['Title']}")
    #         d['Site']=
            
            d['Links']={}
            d['Scope']=''
            doc=pq(url=f'https://www.scimagojr.com/journalsearch.php?q={d["Sourceid"]}&tip=sid&clean=0')
            links=doc('#question_journal')
            for l in links:
                d['Links'][l.text]=l.attrib['href']

            x=doc('div.fullwidth')[0]
            txt=pq(x).text()
            for c in x.getchildren():
                txt=txt.replace(pq(c).text(),'')
            d['Scope']=txt.strip()
    #         d['Site']=d['Links'].get('Homepage','')
    #         if not d['Site']:
    #             for p in d['Links']:
    #                 d['Site']=d['Links'][p]
    #                 break

    #         print(d)    
    #         return
    except Exception as e:
        print(e)
    with open(f'scimagojr_journal_2022_cs_link.json', 'w') as fp:
        json.dump(info, fp)
    return info;
a=update_link_journal()



19300156903:Foundations and Trends in Machine Learning
18902:IEEE Journal on Selected Areas in Communications
19600166419:Foundations and Trends in Signal Processing
21100831443:Nature Biomedical Engineering
18935:IEEE Wireless Communications
21100886132:Science Robotics
4700152228:Molecular Systems Biology
21101007236:Nature Machine Intelligence
25037:IEEE Transactions on Evolutionary Computation
18933:IEEE Transactions on Wireless Communications
17915:Proceedings of the IEEE
21100274221:IEEE Transactions on Cybernetics
5100155078:Foundations and Trends in Computer Graphics and Vision
17249:IEEE Communications Magazine
19700170610:IEEE Transactions on Smart Grid
144974:Journal of Service Research
15631:International Journal of Information Management
26099:Information Fusion
21101101212:AI Open
23038:ACM Computing Surveys
24254:IEEE Transactions on Pattern Analysis and Machine Intelligence
21101081601:PRX Quantum
19400158592:Mathematical Programming Computation
17339:IEEE Transactions 

In [21]:
import pandas as pd
import numpy as np
from pyquery import PyQuery as pq

def update_reviw_time():
    info=json.load(open('scimagojr_journal_2022_cs_link.json'))
    
    for d in info:
#             if(d.get('ReviewFirst','')!=''):
#                  del d['ReviewFirst']
#                  del d['ReviewFinal']
#             continue
            if ('elsevier' in d['Links'].get('Homepage','')):
                print(f"{d['Sourceid']}:{d['Title']}")# {d['Links'].get('Homepage','')}")
                for Issn in d['Issn'].split(', '):
                    issn2=(f'{Issn[0:4]}-{Issn[4:8]}')
                    try:
                        d['Review']={}
                        doc=pq(f'https://journalinsights.elsevier.com/journals/{issn2}/review_speed')
                        for i in range(1,4):
                            td=doc(f'section.graph-data.linegraph tbody tr:nth-child({i}) td')
                            d['Review'][pq(td[0]).text()]={
                                'First':int(float(pq(td[1]).text())*7),
                                'Final':int(float(pq(td[2]).text())*7)
                            }

                    except  Exception as e:
                        print(e)
                    try:
                        d['Acceptance']={}
                        doc=pq(f'https://journalinsights.elsevier.com/journals/{issn2}/acceptance_rate')
                        for i in range(1,4):
                            td=doc(f'section.graph-data.percentage_bars tbody tr:nth-child({i}) td')
                            d['Acceptance'][pq(td[0]).text()]={
                                'Submission':int(float(pq(td[1]).text())),
                                'Accept':int(float(pq(td[2]).text())),
                                'Rate':int(float(pq(td[3]).text()))
                            }
                        print(d)

                    except  Exception as e:
                        print(e)
                        
            if ('springer2343243242' in d['Links'].get('Homepage','')):
                try:
                        d['Review']={}
                        doc=pq(d['Links']['Homepage'])
                        sel=lambda p:doc(f"dd.app-journal-metrics__details[data-test='"+p+"']").text()
                        td=doc(f"dd.app-journal-metrics__details[data-test='impact-factor-value']")
#                         year=sel('impact-factor-value').split(' ')[1][1:5]
                        year=2020
                        IF=sel('impact-factor-value').split(' ')[0]
                        IF5=sel('five-year-impact-factor-value').split(' ')[0]
                        first=sel('metrics-speed-value').split(' ')[0]
                        final=sel('metrics-acceptance-time-value').split(' ')[0]
                        
                        if(first or final):
                            d['Review'][year]={}
                        if first: d['Review'][year]['First']=int(first)
                        if final: d['Review'][year]['Final']=int(final)
                            
                        print(f"{d['Sourceid']}:{d['Title']}")# {d['Links'].get('Homepage','')}")
                except  Exception as e:
                        print(e)
                        print(d['Links'].get('Homepage',''))
                        
    
    with open(f'scimagojr_journal_2022_cs_link.json', 'w') as fp:
        json.dump(info, fp)
    return info;
a=update_reviw_time()



15631:International Journal of Information Management
list index out of range
list index out of range
12396:Journal of Strategic Information Systems
list index out of range
list index out of range
17645:Computers and Education
list index out of range
list index out of range
16965:Internet and Higher Education
list index out of range
list index out of range
29161:ISPRS Journal of Photogrammetry and Remote Sensing
list index out of range
list index out of range
20893:Transportation Research Part C: Emerging Technologies
list index out of range
list index out of range
18080:Robotics and Computer-Integrated Manufacturing
list index out of range
list index out of range
14966:Journal of Manufacturing Systems
list index out of range
list index out of range
8000153138:Computer Science Review
list index out of range
list index out of range
19900192513:Swarm and Evolutionary Computation
list index out of range
list index out of range
21080:Mechanical Systems and Signal Processing
list index out 

In [ ]:
import pandas as pd
import numpy as np

def update_aut_rank():
    info=json.load(open('scimagojr_journal_2020_cs_link.json'))
    aut_q=pd.read_csv('aut-q-2020.csv')
    aut_q['Qnum']=5-aut_q['Q'].str.replace('Q','').astype(int)
    aut_q_m=aut_q.groupby('ISSN').mean()[['Qnum']]
    aut_q_m['Qnum']='Q'+(5-aut_q_m['Qnum'].round().astype(int)).astype(str)
    aut_q_m.index=aut_q_m.index.str.replace('-','')
    
    for d in info:
       if d['Issn']:
         for issn in d['Issn'].split(', '):
            if issn in aut_q_m.index:
                d['aut']=aut_q_m.loc[issn]['Qnum']
    
    with open(f'scimagojr_journal_2020_cs_link_aut.json', 'w') as fp:
        json.dump(info, fp)
    return info;
a=update_aut_rank()



In [25]:
import pandas as pd
import numpy as np
import json
def update_aut_rank():
    info=json.load(open('scimagojr_journal_2022_cs_link.json'))
    aut_q=pd.read_excel('aut-q-2022.xlsx',header=0)
    
    aut_q_m=aut_q.set_index('ISSN')
    aut_q_m.index=aut_q_m.index.str.replace('-','')
    aut_q_e=aut_q.set_index('EISSN')
    aut_q_e.index=aut_q_e.index.str.replace('-','')
    
    for d in info:
       if d['Issn']:
         for issn in d['Issn'].split(', '):
            if issn in aut_q_m.index:
                d['aut']=str(aut_q_m.loc[issn]['Best Quartile'])
                d['aut-info']=str(aut_q_m.loc[issn]['Quartiles'])
                print(d['aut'],d['aut-info'])
            if issn in aut_q_e.index:
                d['aut']=str(aut_q_e.loc[issn]['Best Quartile'])
                d['aut-info']=str(aut_q_e.loc[issn]['Quartiles'])
                print(d['aut'],d['aut-info'])
    
    with open(f'scimagojr_journal_2022_cs_link_aut.json', 'w') as fp:
        json.dump(info, fp)
    return info;
a=update_aut_rank()



Q1 ENGINEERING, ELECTRICAL & ELECTRONIC - SCIE (Q1); TELECOMMUNICATIONS - SCIE (Q1)
Q1 ENGINEERING, BIOMEDICAL - SCIE (Q1)
Q1 ENGINEERING, BIOMEDICAL - SCIE (Q1)
Q1 ENGINEERING, ELECTRICAL & ELECTRONIC - SCIE (Q1); TELECOMMUNICATIONS - SCIE (Q1); COMPUTER SCIENCE, INFORMATION SYSTEMS - SCIE (Q1); COMPUTER SCIENCE, HARDWARE & ARCHITECTURE - SCIE (Q1)
Q1 ROBOTICS - SCIE (Q1)
Q1 ROBOTICS - SCIE (Q1)
Q1 BIOCHEMISTRY & MOLECULAR BIOLOGY - SCIE (Q1)
Q1 BIOCHEMISTRY & MOLECULAR BIOLOGY - SCIE (Q1)
Q1 COMPUTER SCIENCE, INTERDISCIPLINARY APPLICATIONS - SCIE (Q1); COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE - SCIE (Q1)
Q1 COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE - SCIE (Q1); COMPUTER SCIENCE, THEORY & METHODS - SCIE (Q1)
Q1 ENGINEERING, ELECTRICAL & ELECTRONIC - SCIE (Q1); TELECOMMUNICATIONS - SCIE (Q1)
Q1 ENGINEERING, ELECTRICAL & ELECTRONIC - SCIE (Q1)
Q1 ENGINEERING, ELECTRICAL & ELECTRONIC - SCIE (Q1)
Q1 COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE - SCIE (Q1); COMPUTER SCIENCE, CYBERNETICS - SC

In [43]:
issn2='0168-874X'
doc=pq(url=f'https://journalinsights.elsevier.com/journals/{issn2}/acceptance_rate')


In [45]:
issn2='0168-874X'
d={}
d['Acceptance']={}
doc=pq(url=f'https://journalinsights.elsevier.com/journals/{issn2}/acceptance_rate')
for i in range(1,4):
    td=doc(f'section.graph-data.percentage_bars tbody tr:nth-child({i}) td')
    # print(td)
    d['Acceptance'][pq(td[0]).text()]={
        'Submission':int(float(pq(td[1]).text())),
        'Accept':int(float(pq(td[2]).text())),
        'Rate':int(float(pq(td[3]).text()))
    }
    
d


{'Acceptance': {'2022': {'Submission': 566, 'Accept': 70, 'Rate': 12},
  '2021': {'Submission': 531, 'Accept': 66, 'Rate': 12},
  '2020': {'Submission': 624, 'Accept': 48, 'Rate': 8}}}

In [47]:
import pandas as pd
import numpy as np
import json
def update_acceptace():
    info=json.load(open('scimagojr_journal_2022_cs_link_aut.json'))

    for d in info:
       if d['Issn']:
         d['Acceptance']={}
         for issn in d['Issn'].split(', '):   
            try:
                doc=pq(url=f'https://journalinsights.elsevier.com/journals/{issn}/acceptance_rate')
            except:
                continue
            for i in range(1,4):
                td=doc(f'section.graph-data.percentage_bars tbody tr:nth-child({i}) td')
                # print(td)
                if(pq(td[0]).text()):
                    d['Acceptance'][pq(td[0]).text()]={
                        'Submission':int(float(pq(td[1]).text())),
                        'Accept':int(float(pq(td[2]).text())),
                        'Rate':int(float(pq(td[3]).text()))
                    }

    
    with open(f'scimagojr_journal_2022_cs_link_aut.json', 'w') as fp:
        json.dump(info, fp)
    return info;
a=update_acceptace()

